In [2]:
import tensorflow as tf
import numpy as np

# 张量 精度
维数	阶	   名字    	例子
0-D	    0	标量 scalar	s=1 2 3
1-D	    1	向量 vector	v=[1, 2, 3]
2-D	    2	矩阵 matrix	m=[[1,2,3],[4,5,6],[7,8,9]]
N-D	    N	张量 tensor	t=[[[ 有几个中括号就是几阶张量

张量类型|精度：
- tf.int32(默认), tf.float32, 深度学习足够了
- tf.float64,  强化学习可以用
- tf.bool, tf.string

## 创建张量 (传入list)

In [ ]:
a = tf.constant([1, 5], dtype=tf.int64)  # 指定保存精度
b = tf.cast(a, dtype=tf.int32)  # 转换精度和类型
print(b)

In [ ]:
a = tf.constant([True, True, False])
b = tf.cast(a, dtype=tf.int32)
print(b)

In [ ]:
# 将Numpy数组转为Tensor
a = np.arange(0, 5)
b = tf.convert_to_tensor(a, dtype=tf.int32)
print("a:", a)
print("转为张量:", b)
print("转为Numpy：", b.numpy())

## 填充张量(维度,指定值)
一维直接写个数  二维用[行，列]  多维用[n,m,.K....]

In [ ]:
a = tf.zeros([2, 3])
b = tf.ones(4)
c = tf.fill([2, 2], 9)
d = tf.zeros_like(c)  # tf.xx_like()与tf.zeros(c.shape)等价
print("a:", a)
print("b:", b)
print("c:", c)
print(d)

In [ ]:
a = tf.range(10, delta=2)
print(a)

## 数学运算
对应元素的四则运算(必须维度相同): TensorFlow已经重载了+ − ∗ / // % **运算符
平方与开方: tf.square  tf.sqrt
矩阵乘: tf.matmul(矩阵1, 矩阵2)
其他：exp(x) tf.math.log(x)

## 理解axis
在一个二维张量或数组中，可以通过调整axis等于0或1控制执行维度。
axis=0代表跨行(经度，down),而axis=1代表跨列(纬度，across)

In [5]:
x = tf.constant([[1, 2, 3], [2, 2, 3]])
print("x:", x)
print("所有数的均值:", tf.reduce_mean(x))  # 不指定axis,则所有元素参与计算
print("每一行的和:", tf.reduce_sum(x, axis=1))
print("每行最小值：", tf.reduce_min(x, axis=1))
print("每列最大值:", tf.reduce_max(x, axis=0))
print("每列最大值的索引：", tf.argmax(x, axis=0))

x: tf.Tensor(
[[1 2 3]
 [2 2 3]], shape=(2, 3), dtype=int32)
所有数的均值: tf.Tensor(2, shape=(), dtype=int32)
每一行的和: tf.Tensor([6 7], shape=(2,), dtype=int32)
每行最小值： tf.Tensor([1 2], shape=(2,), dtype=int32)
每列最大值: tf.Tensor([2 2 3], shape=(3,), dtype=int32)
每列最大值的索引： tf.Tensor([1 0 0], shape=(3,), dtype=int64)


## 索引和切片
切片[start:end:step] 可根据需要省略；和列表切片一样，左闭右开

In [8]:
x = tf.random.normal([4,32,32,3])  # 4张32x32大小的彩色图片(RGB)
print(x[0,1,2])  # 取第1张图片，第2行，第3列的像素  x[0][1][2]
x2 = x[1:3, ::2, ::-2]  # 表示取第2 3张图片，隔行采样，逆序隔列采样，所有通道信息
x3 = x[..., 1]  # 读取所有图片的G通道
print(x3.shape, x3.ndim)

tf.Tensor([-0.34981516  0.65928495  0.6753863 ], shape=(3,), dtype=float32)
(4, 32, 32) 3


## 维度变换 | 广播机制

In [ ]:
x = tf.range(96)
x = tf.reshape(x, [2, 4, 4, 3])  # 改变张量的视图(理解方式)，并不会改变张量的存储顺序
x = tf.transpose(x, perm=[1,2,0,3])  # 交换维度(改变存储顺序) perm表示新维度的顺序
x = tf.reshape(x, [16, 6])
x = tf.expand_dims(x, axis=0) # 指定的axis轴前插入新的维度 (为负则是在其后插入)
x = tf.squeeze(x, axis=1)  # axis参数为待删除的维度的索引号
print(x.shape)

In [18]:
a = tf.random.normal([3, 1])
b = tf.random.normal([1, 3])
c = a + b  # 广播机制
print(c.shape)

(3, 3)


## where(条件语句，真返回A，假返回B)
类似三元运算符

In [6]:
a = tf.constant([1, 2, 3, 1, 1])
b = tf.constant([0, 1, 3, 4, 5])
c = tf.where(tf.greater(a, b), a, b)  # 若a>b，返回a对应位置的元素，否则返回b对应位置的元素
print("c：", c)  # [1 2 3 4 5]

c： tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)


# 常用函数

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
"""传入特征与标签：切分传入张量的第一维度，生成输入特征标签对，构建数据集 (适用Numpy和Tensor)"""
features = tf.constant([12, 23, 10, 17])
labels = tf.constant([0, 1, 1, 0])
dataset = tf.data.Dataset.from_tensor_slices((features, labels))  # (输入特征，标签)
for element in dataset:
    print(element)

In [ ]:
"""
函数对指定参数求导，如y=x^2求导
tf.Variable(初始值) 将变量标记为“可训练”，被标记的变量会在反向传播中记录梯度信息。
w = tf.Variable(tf.random.normal([2, 2], mean=0, stddev=1))
"""
with tf.GradientTape() as tape:  # with结构记录计算过程
    w = tf.Variable(tf.constant(tf.cast(range(4), tf.float32)))
    loss = tf.pow(w, 2)
grad = tape.gradient(loss, w)  # 求出张量的梯度=2w (函数，对谁求导)
print(grad)  # tf.Tensor([0. 2. 4. 6.], shape=(4,), dtype=float32)

## 独热编码
独热编码(one-hot encoding) ：在分类问题中，常用独热码做标签，标记类别: 1表示是，0表示非。
0狗尾草鸢尾 1杂色鸢尾 2弗吉尼亚鸢尾；
标签1表示为[0. 1. 0.] 0%可能是0狗尾草鸢尾；100%可能是1杂色鸢尾；0%可能是2弗吉尼亚鸢尾

In [ ]:
labels = tf.constant([1, 0, 2, 1])  # 输入的元素值最小为0，最大为2，共3分类
output = tf.one_hot(labels, depth=3)  # (待转换数据，depth=几分类)
print("result of labels1:", output)

In [ ]:
"""softmax() 使n分类的n个输出(y0,y1,...yn-1)通过softmax()函数符合概率分布；"""
y = tf.constant([1.01, 2.01, -0.66])
y_pro = tf.nn.softmax(y)
print("After softmax, y_pro is:", y_pro)  # y_pro 符合概率分布
print("The sum of y_pro:", tf.reduce_sum(y_pro))  # 通过softmax后，所有概率加起来和为1

## 待优化张量Variable
由于梯度运算会消耗大量的计算资源，而且会自动更新相关参数；
对于不需要的优化的张量，如神经网络的输入X，不需要通过tf.Variable封装；
对于需要计算梯度并优化的张量，如神经网络层的W和b，需要通过tf.Variable包裹以便TensorFlow跟踪相关梯度信息；

In [ ]:
w = tf.Variable(4)
w.assign_sub(1)  # (w要自减的内容) 即w=w-1
print("x:", w)  # 4-1=3

# 随机数生成
同numpy，不过顺序稍微不同
np.random
binomial() 二项分布  beta() Beta分布的样本值
## 正态分布

In [ ]:
# 生成正态分布的随机数，默认均值为0，标准差为1 (维度，mean=均值，stddev=标准差)
print(tf.random.normal([2, 2], mean=0.5, stddev=1))
# 生成截断式正态分布的随机数，取值在(μ-2σ， μ+2σ)之内 (维度，mean=均值，stddev=标准差)
print(tf.random.truncated_normal([2, 2], mean=0.5, stddev=1))

## 均匀分布

In [15]:
# 生成均匀分布随机数 (维度，minval=最小值，maxval=最大值)
print(tf.random.uniform([2, 2], minval=0, maxval=1))

[[0.85719534 0.46885937]
 [0.2352003  0.05984159]]
f: tf.Tensor(
[[0.17577958 0.62613165]
 [0.6155919  0.31002963]], shape=(2, 2), dtype=float32)
